<a href="https://colab.research.google.com/github/noelmt7/LAB-4/blob/main/LAB4_2348538.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import threading
import time
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df=pd.read_csv('/content/Groceries_dataset - Groceries_dataset (1).csv')

In [3]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [5]:
df.shape

(38765, 3)

In [6]:
all_transactions = [transaction[1]['itemDescription'].tolist()
for transaction in list(df.groupby(['Member_number', 'Date']))]

In [7]:
len(all_transactions)

14963

In [8]:
all_transactions[0:10]

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables'],
 ['frankfurter', 'curd'],
 ['sausage', 'whole milk', 'rolls/buns'],
 ['whole milk', 'soda'],
 ['beef', 'white bread'],
 ['frankfurter', 'soda', 'whipped/sour cream']]

In [9]:
trans_encoder = TransactionEncoder()
trans_encoder_matrix = trans_encoder.fit(all_transactions).transform(all_transactions)
trans_encoder_matrix_int = trans_encoder_matrix.astype(int)
trans_encoder_matrix_df = pd.DataFrame(trans_encoder_matrix_int, columns=trans_encoder.columns_)

In [10]:
trans_encoder_matrix_df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
apiThread=threading.Event()
fpThread=threading.Event()

In [12]:
minSupportInput = input("Enter the MIN Support value: ")
minSupport = float(minSupportInput)
minThreshInput = input("Enter the MIN Threshold value: ")
minThresh = float(minThreshInput)

Enter the MIN Support value: 0.2
Enter the MIN Threshold value: 1.0


In [13]:
executionTime = [0,0]

In [14]:
def runapi():
  stime = time.time()
  rule_items = apriori(trans_encoder_matrix_df, min_support = min_support, use_colnames = True)
  executionTime[0] = time.time()-stime
  apiThread.set()
  association_rules(rule_items, minThresh, "Apriori Association")


In [15]:
def runfp():
  stime = time.time()
  rule_items = fpgrowth(trans_encoder_matrix_df, min_support= minSupport, use_colnames=True)
  executionTime[1] = time.time()-stime
  fpthread.set()

In [16]:
apthread=threading.Thread(target=runapi)
fpthread=threading.Thread(target=runfp)

In [17]:
apicompleted=threading.Event()
fpcompleted=threading.Event()


In [18]:
# apthread.start()
# fpthread.start()
# apiThread.wait()
# fpThread.wait()
# apthread.join()
# fpthread.join()